# 생태통로 반경 2.3km 이내 로드킬 빈도

## 패키지

In [3]:
!pip install pymysql
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from haversine import haversine

## 데이터 로드

In [5]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [6]:
def sql_to_df(table_name):
    sql = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql(sql, db_connection)
    return df

In [7]:
main = sql_to_df("MainData")
roadkill = sql_to_df("로드킬_2021")

In [8]:
main.head()

,번호,위도_십진법,경도_십진법,생태통로타입,차선_수,등산객이용빈도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",이용확인 종 수,면적_m^2,너비_m,길이_m
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,NaN,NaN
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,NaN,NaN
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0


In [9]:
main.shape

(497, 12)

In [10]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   번호                       497 non-null    int64  
 1   위도_십진법                   497 non-null    float64
 2   경도_십진법                   497 non-null    float64
 3   생태통로타입                   497 non-null    int64  
 4   차선_수                     497 non-null    int64  
 5   등산객이용빈도                  497 non-null    int64  
 6   생태통로효율성                  497 non-null    int64  
 7   이용확인종,이용빈도높은종,멸종위기종이용빈도  211 non-null    object 
 8   이용확인 종 수                 497 non-null    int64  
 9   면적_m^2                   240 non-null    float64
 10  너비_m                     240 non-null    float64
 11  길이_m                     240 non-null    float64
dtypes: float64(5), int64(6), object(1)
memory usage: 46.7+ KB


In [11]:
roadkill.head()

,본부명,지사명,노선명,구간,방향,5km,발생건수,위도,경도
0,수도권,경기광주,제2중부선,330~335,통영,330.0,5.0,37.283059,127.385590
1,수도권,경기광주,제2중부선,335~340,하남,335.0,3.0,37.315325,127.347275
2,수도권,경기광주,중부선,330~335,통영,330.0,3.0,37.283193,127.384920
3,수도권,경기광주,중부선,340~345,통영,340.0,3.0,37.353642,127.320539
4,수도권,경기광주,중부선,325~330,하남,325.0,3.0,37.248957,127.421370


In [12]:
roadkill.shape

(108, 9)

In [13]:
roadkill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   본부명     108 non-null    object 
 1   지사명     108 non-null    object 
 2   노선명     108 non-null    object 
 3   구간      108 non-null    object 
 4   방향      108 non-null    object 
 5   5km     108 non-null    float64
 6   발생건수    108 non-null    float64
 7   위도      108 non-null    float64
 8   경도      108 non-null    float64
dtypes: float64(4), object(5)
memory usage: 7.7+ KB


## 거리 계산

In [14]:
# A-B 지점 간 거리(km) 계산 함수
def distance(A_lat, A_long, B_lat, B_long): # A 지점 위도 및 경도, B 지점 위도 및 경도
    A = (A_lat, A_long)
    B = (B_lat, B_long)
    dist = haversine(A, B, unit='km')
    return dist

In [16]:
# test
distance(main.loc[0, '위도_십진법'], main.loc[0, '경도_십진법'], roadkill.loc[0, '위도'], roadkill.loc[0, '경도']) 

222.27619457368831

In [17]:
dist_list = []
i_list = []
j_list = []
for i, m_row in main.iterrows():
    for j, rk_row in roadkill.iterrows():
        dist_list.append(distance(m_row['위도_십진법'], m_row['경도_십진법'], rk_row['위도'], rk_row['경도']))
        i_list.append(i)
        j_list.append(j)

In [18]:
print(len(dist_list))
print(dist_list)

53676
[222.27619457368831, 226.6734919260934, 222.3068369635947, 231.43547745577578, 217.7656020083232, 222.3068369635947, 241.3158237146603, 244.66633482380036, 263.78774978853323, 245.3785172642111, 255.5174644431001, 253.63541862423546, 269.6394634416986, 270.09921985330976, 222.62930260870553, 225.94033853525244, 220.67039676004185, 246.17382667986294, 250.7387944079889, 254.5821734792155, 257.41662248077836, 268.66816915056455, 276.78330845850064, 220.67039676004185, 229.23131236142535, 246.17382667986294, 268.66816915056455, 272.7893260535706, 276.78330845850064, 145.55563588301027, 141.9474085075242, 121.92123577409957, 187.96515951106431, 194.96251918841043, 188.46953444531428, 184.0438439871587, 156.23808179558003, 184.34974500085963, 193.49592355941564, 212.85921996107436, 151.80861230170487, 160.18105973693585, 165.44117482825072, 193.49592355941564, 198.42041414242559, 163.41791644219902, 154.36942591134726, 137.74392128463612, 150.11923592226918, 146.78008572593174, 133.00

In [19]:
dist_df = pd.DataFrame({'main_idx': i_list,
                        'roadkill_idx': j_list,
                        'distance': dist_list})
dist_df

,main_idx,roadkill_idx,distance
0,0,0,222.276195
1,0,1,226.673492
2,0,2,222.306837
3,0,3,231.435477
4,0,4,217.765602
...,...,...,...
53671,496,103,100.572902
53672,496,104,93.522200
53673,496,105,100.873352
53674,496,106,202.324231


## 반경 2.3km 이내 로드킬이 발생한 건

In [21]:
within = dist_df[dist_df['distance'] <= 2.3] # 반경 2.3km 이내
within

,main_idx,roadkill_idx,distance
473,4,41,1.474966
3218,29,86,1.217189
3652,33,88,0.397620
7390,68,46,1.878145
7500,69,48,1.866237
10811,100,11,1.553199
10920,101,12,1.769068
11995,111,7,2.292573
12290,113,86,1.688272
14050,130,10,1.643592


In [22]:
main.iloc[within['main_idx']]

,번호,위도_십진법,경도_십진법,생태통로타입,차선_수,등산객이용빈도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",이용확인 종 수,면적_m^2,너비_m,길이_m
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0
29,1462,35.36617,127.12414,1,1,0,2,None,0,NaN,NaN,NaN
33,1467,35.54294,127.69214,1,1,0,3,None,0,NaN,NaN,NaN
68,1374,36.48645,127.07944,0,1,1,1,고라니 너구리 고양이,3,623.28,9.80,63.6
69,1375,36.49499,127.14486,1,1,1,0,고양이,1,NaN,NaN,NaN
100,1518,37.50025,129.09139,0,1,0,3,None,0,1665.00,37.00,45.0
101,1362,37.68722,128.91671,0,1,0,0,고라니 너구리,2,NaN,NaN,NaN
111,1561,37.37799,126.98882,0,0,5,1,None,0,1711.05,91.50,18.7
113,1118,35.35865,127.11915,0,0,0,2,고라니 멧돼지 두꺼비,3,737.04,24.90,29.6
130,1052,37.45336,126.85687,0,2,5,3,None,0,1809.50,32.90,55.0


In [23]:
roadkill.iloc[within['roadkill_idx']]

,본부명,지사명,노선명,구간,방향,5km,발생건수,위도,경도
41,충북,진천,중부선,260~265,하남,260.0,5.0,36.708802,127.437539
86,광주전남,남원,광주대구선,30~35,광주,30.0,3.0,35.362999,127.136988
88,광주전남,남원,광주대구선,90~95,대구,90.0,3.0,35.543314,127.696511
46,대전충남,공주,당진대전선,60~65,당진,60.0,3.0,36.484786,127.058534
48,대전충남,공주,당진대전선,70~75,대전,70.0,3.0,36.497571,127.165489
11,강원,강릉,동해선,15~20,속초,15.0,3.0,37.486292,129.090703
12,강원,강릉,동해선,50~55,속초,50.0,3.0,37.690838,128.897132
7,수도권,동서울,수도권제1순환선,120~125,일산,120.0,4.0,37.390300,127.009635
86,광주전남,남원,광주대구선,30~35,광주,30.0,3.0,35.362999,127.136988
10,수도권,시흥,제2경인선,20~25,안양,20.0,3.0,37.439880,126.849232


## 로드킬 빈도

In [24]:
main['roadkill_freq'] = 0
main

,번호,위도_십진법,경도_십진법,생태통로타입,차선_수,등산객이용빈도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",이용확인 종 수,면적_m^2,너비_m,길이_m,roadkill_freq
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,NaN,NaN,0
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,NaN,NaN,0
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,0
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,0
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,0,2,0,0,고라니 멧돼지,2,674.31,13.30,50.7,0
493,1087,36.71722,127.41778,0,1,5,0,고라니 멧돼지,2,526.32,13.60,38.7,0
494,1088,36.71222,127.42528,0,1,5,0,고라니 멧돼지,2,674.31,13.30,50.7,0
495,1091,36.76750,127.47333,0,0,0,0,멧돼지 고라니,2,140.00,7.00,20.0,0


In [25]:
roadkill[roadkill['발생건수'] == 0]

,본부명,지사명,노선명,구간,방향,5km,발생건수,위도,경도


In [26]:
for i, w_row in within.iterrows():
    if main.loc[w_row['main_idx'], 'roadkill_freq'] != 0:
        main.loc[w_row['main_idx'], 'roadkill_freq'] += roadkill.loc[w_row['roadkill_idx'], '발생건수']
    else:
        main.loc[w_row['main_idx'], 'roadkill_freq'] = roadkill.loc[w_row['roadkill_idx'], '발생건수']

In [27]:
main['roadkill_freq']

0      0
1      0
2      0
3      0
4      5
      ..
492    5
493    5
494    5
495    0
496    0
Name: roadkill_freq, Length: 497, dtype: int64

In [28]:
main[main['roadkill_freq'] != 0]

,번호,위도_십진법,경도_십진법,생태통로타입,차선_수,등산객이용빈도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",이용확인 종 수,면적_m^2,너비_m,길이_m,roadkill_freq
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0,5
29,1462,35.36617,127.12414,1,1,0,2,None,0,NaN,NaN,NaN,3
33,1467,35.54294,127.69214,1,1,0,3,None,0,NaN,NaN,NaN,3
68,1374,36.48645,127.07944,0,1,1,1,고라니 너구리 고양이,3,623.28,9.80,63.6,3
69,1375,36.49499,127.14486,1,1,1,0,고양이,1,NaN,NaN,NaN,3
100,1518,37.50025,129.09139,0,1,0,3,None,0,1665.00,37.00,45.0,3
101,1362,37.68722,128.91671,0,1,0,0,고라니 너구리,2,NaN,NaN,NaN,3
111,1561,37.37799,126.98882,0,0,5,1,None,0,1711.05,91.50,18.7,4
113,1118,35.35865,127.11915,0,0,0,2,고라니 멧돼지 두꺼비,3,737.04,24.90,29.6,3
130,1052,37.45336,126.85687,0,2,5,3,None,0,1809.50,32.90,55.0,3


In [38]:
len(main[main['roadkill_freq'] != 0])

27

## 최종 확인

In [29]:
main

,번호,위도_십진법,경도_십진법,생태통로타입,차선_수,등산객이용빈도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",이용확인 종 수,면적_m^2,너비_m,길이_m,roadkill_freq
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,NaN,NaN,0
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,NaN,NaN,0
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,0
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN,0
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,1089,36.71278,127.42056,0,2,0,0,고라니 멧돼지,2,674.31,13.30,50.7,5
493,1087,36.71722,127.41778,0,1,5,0,고라니 멧돼지,2,526.32,13.60,38.7,5
494,1088,36.71222,127.42528,0,1,5,0,고라니 멧돼지,2,674.31,13.30,50.7,5
495,1091,36.76750,127.47333,0,0,0,0,멧돼지 고라니,2,140.00,7.00,20.0,0


### 최종 업로드 할 데이터프레임

In [32]:
main['번호'].nunique()

497

In [33]:
final = main[['번호', 'roadkill_freq']]

In [34]:
final.columns = ['번호', '주변 로드킬 빈도']

In [35]:
final

,번호,주변 로드킬 빈도
0,1495,0
1,1277,0
2,1278,0
3,1279,0
4,1090,5
...,...,...
492,1089,5
493,1087,5
494,1088,5
495,1091,0


In [36]:
final.dtypes

번호           int64
주변 로드킬 빈도    int64
dtype: object

## RDS에 데이터프레임 업로드

In [37]:
final.to_sql(name='MainData_inner_예림', con=db_connection, if_exists='replace', index=False) # if_exists options : append, fail, replace